In [40]:
import pandas as pd
import numpy as np
import requests
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

- 0. kaggle titanic raw data -> 70% (modeling - offline) / 30% (testing - online) spliting for holdout
- 1. kaggle titanic holdout -> modeling
- 2. import model to cloud function
- 3. predict from cloud function

# 1. Retrieve datasets from Kaggle via API

In [13]:
!pip install kaggle

In [17]:
from sklearn.datasets import load_iris
data = load_iris()

In [19]:
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [21]:
data['target']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [26]:
X_df = pd.DataFrame(data['data'], columns=data['feature_names'])

In [27]:
X_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [28]:
y = data['target']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

len(y_train)

X_test.shape

len(y_test)

In [37]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [39]:
clf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [41]:
filename = 'iris_classifer.sav'
pickle.dump(clf, open(filename, 'wb'))

# ------------- Cloud function --------

In [ ]:
from google.cloud import storage

bucket_name = 'asia-northeast1-iris-pipeli-d3174245-bucket'

storage_client = storage.Client()

bucket=storage_client.get_bucket(bucket_name)

blobs = bucket.list_blobs(prefix='iris_model/iris_classifer.sav')
for blob in blobs:
    model_str_downloaded = blob.download_as_string()
    pickled = pickle.loads(model_str_downloaded)

    

pickled.predict(X_test)

In [111]:
X_test.head(1).values[0].reshape(-1,1)

array([[6.1],
       [2.8],
       [4.7],
       [1.2]])

In [118]:
[list(X_test.head(1).values[0])]

[[6.1, 2.8, 4.7, 1.2]]

In [116]:
pickled.predict([list(X_test.head(1).values[0])])

array([1])

In [70]:
from flask import jsonify
from google.cloud import storage

bucket_name = 'asia-northeast1-iris-pipeli-d3174245-bucket'

def hello_world(request):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """
    
    storage_client = storage.Client()

    bucket=storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix='iris_model/iris_classifer.sav')
    for blob in blobs:
        model_str_downloaded = blob.download_as_string()
        pickled = pickle.loads(model_str_downloaded)



    
    
    request_json = request.get_json()
    
    if request_json and 'predict' in request_json:
        return jsonify(pickled.predict(request_json['predict']))
    else:
        return jsonify({"reponse": "error"})


[6.1, 2.8, 4.7, 1.2]

In [99]:
from flask import jsonify
from google.cloud import storage
import pickle
from sklearn.ensemble import RandomForestClassifier

bucket_name = 'asia-northeast1-iris-pipeli-d3174245-bucket'

def hello_world(request):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """
    
    storage_client = storage.Client()

    bucket=storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix='iris_model/iris_classifer.sav')
    for blob in blobs:
        model_str_downloaded = blob.download_as_string()
        pickled = pickle.loads(model_str_downloaded)



    
    
    request_json = request.get_json()
    
    
    X = np.array(request_json['predict']).reshape(-1,1)
    
    if request_json and 'predict' in request_json:
        return jsonify(pickled.predict(X))
    else:
        return jsonify({"reponse": "error"})


array([[6.1],
       [2.8],
       [4.7],
       [1.2]])

In [135]:
X = [list(X_test.head(1).values[0])]

In [136]:
X

[[6.1, 2.8, 4.7, 1.2]]

In [140]:
{'result': int(pickled.predict(X)[0])}

{'result': 1}

In [156]:
def predict(X):

    model_url = 'https://us-central1-contxtsio-267105.cloudfunctions.net/function-1'

    payload = {
        'predict': X
    }

    result = requests.post(model_url, json=payload)
    return result.json()['result']


In [157]:
list(X_test.values[0])

[6.1, 2.8, 4.7, 1.2]

In [158]:
for obs_np in X_test.values:
    obs = list(obs_np)
    print(predict(obs))

1
0
2
1
1
0
1
2
1
1
2
0
0
0
0
1
2
1
1
2
0
2
0
2
2
2
2
2
0
0
0
0
1
0
0
2
1
0
0
0
2
1
1
0
0
1
1
2
1
2


In [144]:
result.json()

{'result': 1}